In [4]:
import warnings
import flwr as fl
import numpy as np
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

import utils

#COMPRESSION MODELS = {'stc','dgc','sgd','none'}
Compression_model = 'stc'
Print_bits = True

device = 'cpu'

def get_bits(T, compression_method, approx=False):
    """
    Returns the number of bits that are required to communicate the Tensor T, which was compressed with compresion_method
    """

    B_val = {"none" : 32, "dgc" : 32, "stc" : 1, "sgd" : 1}[compression_method]

    # dense methods
    if compression_method in ["none", "sgd"]:
        k = T.numel()
        B_pos = 0

    # sparse methods non-optimal encoding
    elif compression_method in ["dgc"]:
        k = torch.sum(T!=0.0).item()
        B_pos = 16

    # sparse methods golomb encoding
    elif compression_method in ["stc"]:
        k = torch.sum(T!=0.0).item()
        N = T.numel()

        q = (k+1)/(N+1)
        golden = (np.sqrt(5)+1)/2

        if q == 1:
            return B_val*T.numel()
        if q == 0:
            return 0

        b_star = 1 + np.floor(np.log2(np.log(golden-1)/np.log(1-q)))

        if approx:
            B_pos = b_star + 1/(1-(1-q)**(2**b_star)) + 1
        else:
            idc = torch.nonzero(T.view(-1))
            distances = idc[:]-torch.cat([torch.Tensor([[-1]]).long().to("cuda"),idc[:-1]])
            B_pos = torch.mean(torch.ceil(distances.float()/2**b_star)).item()+(b_star+1)

    b_total = (B_pos+B_val)*k
    f = open("bitsEnviados.txt", "a")
    f.write(str(b_total)+"\n")
    f.close()
    return b_total

def approx_v(T, p, frac):
    if frac < 1.0:
        n_elements = T.numel()
        n_sample = min(int(max(np.ceil(n_elements * frac), np.ceil(100/p))), n_elements)
        n_top = int(np.ceil(n_sample*p))

        if n_elements == n_sample:
            i = 0
        else:
            i = np.random.randint(n_elements-n_sample)

        topk, _ = torch.topk(T.flatten()[i:i+n_sample], n_top)
        if topk[-1] == 0.0 or topk[-1] == T.max():
            return approx_v(T, p, 1.0)
    else:
        n_elements = T.numel()
        n_top = int(np.ceil(n_elements*p))
        topk, _ = torch.topk(T.flatten(), n_top)

    return topk[-1], topk

#STC COMPRESSION
def STC(param, p, aprox):
    seq = torch.as_tensor(param[0], dtype= float, device='cpu')
    label = torch.as_tensor(param[1], dtype= float, device='cpu')
    seq = seq.type(torch.FloatTensor)
    label = label.type(torch.FloatTensor)
    T = [seq,label]
    tensor = [seq,label]
    for x in range(len(T)):
        T_abs = torch.abs((T[x]))
        v, topk = approx_v(T_abs, p, aprox)
        mean = torch.mean(topk)  
        out_ = torch.where(T[x] >= v, mean, torch.Tensor([0.0]).to(device))
        out = torch.where(T[x] <= -v, -mean, out_)
        tensor[x] = out
        T[x] = out
        T[x] = T[x].cpu().detach().numpy()
    if Print_bits:
        print(get_bits(tensor[0],'stc',aprox) + get_bits(tensor[1],'stc',aprox))
    return (T[0],T[1])

#DGC COMPRESSION
def DGC(param, p, aprox):
    '''
    "Deep Gradient Compression: Reducing the communication Bandwidth for Distributed Training, Lin et al."
    '''
    if p >= 1.0:
        return param
    seq = torch.as_tensor(param[0], dtype= float, device='cuda')
    label = torch.as_tensor(param[1], dtype= float, device='cuda')
    seq = seq.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    T = [seq,label]
    tensor = [seq,label]
    for x in range(len(T)):
        T_abs = torch.abs(T[x])
        v, _ = approx_v(T_abs, p, aprox)
        out = torch.where(T_abs >= v, T[x], torch.Tensor([0.0]).to(device))
        tensor[x] = out
        T[x] = out
        T[x] = T[x].cpu().detach().numpy()
    if Print_bits:
        print(get_bits(tensor[0],'dgc',aprox) + get_bits(tensor[1],'dgc',aprox))
    return (T[0],T[1])

#NO COMPRESSION
def none(param, p, aprox):
    '''
    Identity
    '''
    seq = torch.as_tensor(param[0], dtype= float, device='cuda')
    label = torch.as_tensor(param[1], dtype= float, device='cuda')
    seq = seq.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    tensor = (seq,label)
    if Print_bits:
        print(get_bits(tensor[0],'none',aprox) + get_bits(tensor[1],'none',aprox))
    return param

#SGD COMPRESSION
def SGD(param,p,aprox):
    """
    signSGD: Compressed Optimisation for non-convex Problems, Bernstein et al.

    """
    seq = torch.as_tensor(param[0], dtype= float, device='cuda')
    label = torch.as_tensor(param[1], dtype= float, device='cuda')
    seq = seq.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    T = [seq,label]
    tensor = [seq,label]
    for x in range(len(T)):
        T[x] = T[x].sign()
        tensor[x] = T[x]
        T[x] = T[x].cpu().detach().numpy()
    if Print_bits:
        print(get_bits(tensor[0],'sgd',aprox) + get_bits(tensor[1],'sgd',aprox))
    return (T[0],T[1])


# Load MNIST dataset from https://www.openml.org/d/554
(X_train, y_train), (X_test, y_test) = utils.load_mnist()

# Split train set into 10 partitions and randomly use one for training.
partition_id = np.random.choice(10)
(X_train, y_train) = utils.partition(X_train, y_train, 10)[partition_id]

# Create LogisticRegression Model
model = LogisticRegression(
    penalty="l2",
    max_iter=1,  # local epoch
    warm_start=True,  # prevent refreshing weights when fitting
)
# Setting initial parameters, akin to model.compile for keras models
utils.set_initial_params(model)

#compress param model
class MnistClient(fl.client.NumPyClient):
    def get_parameters(self): # type: ignore
        return utils.get_model_parameters(model)
    
    def fit(self, parameters, config): # type: ignore
        utils.set_model_params(model, parameters)
        # Ignore convergence failure due to low local epochs
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)
            #-------------------------TRANSFORM----------------
            if Compression_model == 'stc':
                T = STC(utils.get_model_parameters(model),0.25,0.8)
            elif Compression_model == 'dgc':
                T = DGC(utils.get_model_parameters(model),0.25,0.8)
            elif Compression_model == 'sgd':
                T = SGD(utils.get_model_parameters(model),0.25,0.8)
            elif Compression_model == 'none':
                T = none(utils.get_model_parameters(model),0.25,0.8)
                
            utils.set_model_params(model, T)    
        
        print(f"Training finished for round {config['rnd']}")
        return utils.get_model_parameters(model), len(X_train), {}

    def evaluate(self, parameters, config): # type: ignore
        utils.set_model_params(model, parameters)
        loss = log_loss(y_test, model.predict_proba(X_test))
        accuracy = model.score(X_test, y_test)
        return loss, len(X_test), {"accuracy": accuracy}
    
fl.client.start_numpy_client("localhost:8080", client=MnistClient())


DEBUG flower 2022-02-21 13:47:04,518 | connection.py:36 | ChannelConnectivity.IDLE
INFO flower 2022-02-21 13:47:04,520 | app.py:61 | Opened (insecure) gRPC connection
DEBUG flower 2022-02-21 13:47:04,520 | connection.py:36 | ChannelConnectivity.READY


10505.71301540864
Training finished for round 1
10688.90352291282
Training finished for round 2
10149.730085754749
Training finished for round 3
10316.011154703214
Training finished for round 4


DEBUG flower 2022-02-21 13:47:06,870 | connection.py:68 | Insecure gRPC channel closed
INFO flower 2022-02-21 13:47:06,870 | app.py:72 | Disconnect and shut down


10465.763155816605
Training finished for round 5
